**Importing the necessary python libraries**

Will continue to update my kernel as and when i find some time and do more analysis...

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
from datetime import date

**Importing matplotlib and seaborn**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

**Loading the dataset to the dataframe (df)**

In [ ]:
df = pd.read_csv('../input/seattleWeather_1948-2017.csv')

**Initial data cleanup and analysis** ( I used a heatmap... you can use other ways to find null values)

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
#Converting the date to datetime object
df['DATE'] = pd.to_datetime(df['DATE'], errors = 'coerce')

In [ ]:
df['DAY'] = df['DATE'].dt.weekday_name
df['DAY_OF_WEEK'] = df['DATE'].apply(date.isoweekday)
df['MONTH'] = df['DATE'].dt.month
df['YEAR'] = df['DATE'].dt.year
df['RAIN'] = df['RAIN'].apply(lambda x: 1 if x == True else 0)

In [ ]:
df.head()

**Total number of rainy days in the last 3 years by month..**

In [ ]:
plt.figure(figsize=(12,5))
ax = sns.countplot(data = df[(df['RAIN'] == True) & (df['YEAR'] >= 2015)], x='MONTH', hue='YEAR')

**Checking the number of rainy days for last 17 years... (looks like the average is around 157mm/year**

In [ ]:
int(df[(df['RAIN'] == True) & (df['YEAR'] >= 2000)].groupby('YEAR').count()['DAY'].mean())

In [ ]:
plt.figure(figsize=(12,5))
ax = sns.countplot(data = df[(df['RAIN'] == True) & (df['YEAR'] >= 2000)], x='YEAR')
plt.tight_layout()

Let's model the forecast with a decision tree classifier

In [ ]:
g = sns.lmplot(x='TMIN', y= 'PRCP', data=df, fit_reg=False, hue='RAIN', size=8)

In [ ]:
g = sns.lmplot(x='TMAX', y= 'PRCP', data=df, fit_reg=False, hue='RAIN', size=8)

In [ ]:
g = sns.lmplot(x='TMIN', y= 'TMAX', data=df, fit_reg=False, hue='RAIN', size=8)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['DATE', 'DAY', 'YEAR', 'DAY_OF_WEEK', 'RAIN'],axis=1), 
                                                    df['RAIN'], test_size=0.30)

In [ ]:
y_train = y_train.fillna(y_train.mean())
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

In [ ]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

In [ ]:
logpredictions = logmodel.predict(X_test)

In [ ]:
print(accuracy_score(y_test,logpredictions))

In [ ]:
print(confusion_matrix(y_test,logpredictions))

In [ ]:
print(classification_report(y_test,logpredictions))

** Trying a decision tree now**

In [ ]:
dtree = DecisionTreeClassifier()

In [ ]:
dtree.fit(X_train,y_train)

In [ ]:
dtree_predictions = dtree.predict(X_test)

In [ ]:
print(accuracy_score(y_test,dtree_predictions))

In [ ]:
print(classification_report(y_test,dtree_predictions))

In this case, a decision tree performed better than a logistic regression algorithm.

In [ ]:
print(confusion_matrix(y_test,dtree_predictions))